# Simulation de la ligne HERMES avec PyOptiX

In [1]:
import ctypes
import sys, os
sys.path.append("../..")
import numpy as np
from scipy.constants import h, c, eV, nano, milli, degree, pi
import pyoptix
from pyoptix.classes import (Beamline, Source, OpticalElement, Diagram, RecordingMode, ToroidalMirror, 
                             PlaneHoloGrating, CylindricalMirror, ConicCylindricalMirror, PlaneGrating,
                             PlaneFilm, PlaneMirror)

intialzing SR library
OptiX library initialized


In [2]:
hc = h*c/nano/eV
pyoptix.output_notebook()

Loading BokehJS ...

## Définition des optiques

In [3]:
Hermes = Beamline()
onduleur_BE = Source(element_type="GaussianSource", name="S_ONDUL_BE")
onduleur_HE = Source(element_type="GaussianSource", name="S_ONDUL_HE")
pupille = PlaneFilm(name="pupille")
m1a = PlaneMirror(name="M1A")
m1b = ToroidalMirror(name="M1B")
m1c = ToroidalMirror(name="M1C")
focalisation_horizontale = PlaneFilm(name="foca")
f = PlaneFilm(name="f")
reseau_450 = PlaneHoloGrating(name="Reseau_450")
reseau_600 = PlaneHoloGrating(name="Reseau_600")
reseau_200 = PlaneHoloGrating(name="Reseau_200")
reseau_300 = PlaneHoloGrating(name="Reseau_300")
reseau_MCA = PlaneGrating(name="Reseau_1800Multi")
m2 = PlaneMirror(name="plan")
m3 = ToroidalMirror(name="M3tor")
m4 = ToroidalMirror(name="M4")
plan_intermediaire = PlaneFilm(name="planint")
fente = PlaneFilm(name="Fente")
m5 = CylindricalMirror(name="M5CylConCave")
foc_intermediaire = PlaneFilm(name="FocIntHo")
diagnostic = PlaneFilm(name="Diag")
m6 = ConicCylindricalMirror(name="M6Cc")
m7 = ConicCylindricalMirror(name="M7Cc")
peem = PlaneFilm(name="PEEM")
stxm = PlaneFilm(name="STXM")

In [4]:
branches = [[m5, foc_intermediaire, m6, m7, peem],
            [m4, stxm]]
for ond, alias_ond in [(onduleur_BE, "BE"),(onduleur_BE, "HE")]:
    for reseau in [reseau_200, reseau_300, reseau_450, reseau_600, reseau_MCA]:
        for branch, branch_name in zip(branches, ["PEEM", "STXM"]):
            chain_name = f"{alias_ond}_{branch_name}_{reseau.name.split('_')[1]}"
            if alias_ond == "BE":
                second_m1 = m1b
            else:
                second_m1 = m1c
            Hermes.chains[chain_name] = [ond,
                                         pupille, m1a, second_m1, focalisation_horizontale, f, 
                                         reseau, m2, m3,
                                         plan_intermediaire, fente] + branch
print(Hermes.chains)
Hermes.active_chain = "BE_PEEM_200"

Chaîne BE_PEEM_200:
	S_ONDUL_BE -> pupille -> M1A -> M1B -> foca -> f -> Reseau_200 -> plan -> M3tor -> planint -> Fente -> M5CylConCave -> FocIntHo -> M6Cc -> M7Cc -> PEEM 
Chaîne BE_STXM_200:
	S_ONDUL_BE -> pupille -> M1A -> M1B -> foca -> f -> Reseau_200 -> plan -> M3tor -> planint -> Fente -> M4 -> STXM 
Chaîne BE_PEEM_300:
	S_ONDUL_BE -> pupille -> M1A -> M1B -> foca -> f -> Reseau_300 -> plan -> M3tor -> planint -> Fente -> M5CylConCave -> FocIntHo -> M6Cc -> M7Cc -> PEEM 
Chaîne BE_STXM_300:
	S_ONDUL_BE -> pupille -> M1A -> M1B -> foca -> f -> Reseau_300 -> plan -> M3tor -> planint -> Fente -> M4 -> STXM 
Chaîne BE_PEEM_450:
	S_ONDUL_BE -> pupille -> M1A -> M1B -> foca -> f -> Reseau_450 -> plan -> M3tor -> planint -> Fente -> M5CylConCave -> FocIntHo -> M6Cc -> M7Cc -> PEEM 
Chaîne BE_STXM_450:
	S_ONDUL_BE -> pupille -> M1A -> M1B -> foca -> f -> Reseau_450 -> plan -> M3tor -> planint -> Fente -> M4 -> STXM 
Chaîne BE_PEEM_600:
	S_ONDUL_BE -> pupille -> M1A -> M1B -> foca -> f 

## Définition des paramètres des optiques

### Onduleur basse énergie

In [5]:
onduleur_BE.nrays = 5000
onduleur_BE.sigma_x = 1.74807e-4
onduleur_BE.sigma_y = 1.477385e-5
onduleur_BE.sigma_x_div = 3.17065e-3*degree # rad
onduleur_BE.sigma_y_div = 2.782436e-3*degree # rad
# Note les dimension sourcees sont recalculées à l'alignement
onduleur_BE.next = pupille

### Onduleur haute énergie

In [6]:
onduleur_HE.nrays = 5000
onduleur_HE.sigma_x = 1.74807e-4
onduleur_HE.sigma_y = 1.477385e-5
onduleur_HE.sigma_x_div = 3.17065e-3*degree # rad
onduleur_HE.sigma_y_div = 2.782436e-3*degree # rad
# Note les dimension sourcees sont recalculées à l'alignement
onduleur_HE.next = pupille

### Pupille

In [7]:
pupille.distance_from_previous = 20
pupille.recording_mode = RecordingMode.recording_output
pupille.next = m1a

### M1a 

In [8]:
m1a.distance_from_previous = 0.4783
m1a.theta = 2.5*degree # rad
m1a.phi = -90*degree # rad
m1a.next = m1b

### M1b 

In [9]:
m1b.distance_from_previous = 3.2096
m1b.theta = 2.5*degree # rad
m1b.phi = 180*degree # rad
m1b.minor_curvature = 1/1.802572 # m-1
m1b.major_curvature = 1/126.7433 # m-1
m1b.next = focalisation_horizontale

### M1c

In [10]:
m1c.distance_from_previous = 0.9791
m1c.theta = 1.2*degree # rad
m1c.phi = 180*degree # rad
m1c.minor_curvature = 1/0.8870043 # m-1
m1c.major_curvature = 1/229.5203 # m-1

### Plan de focalisation_horizontale

In [11]:
focalisation_horizontale.phi = -90*degree # rad
focalisation_horizontale.distance_from_previous = 3.2096
focalisation_horizontale.recording_mode = RecordingMode.recording_output
focalisation_horizontale.next = reseau_450

### Réseau 200 tpmm

In [12]:
reseau_200.line_density = 200/milli
reseau_200.inverse_distance1=0.1439452
reseau_200.inverse_distance2=0.1111111
reseau_200.elevation_angle1=np.arccos(0.7986355)

### Réseau 300 tpmm 

In [13]:
reseau_300.line_density = 300/milli
reseau_300.inverse_distance1=0.3247547
reseau_300.inverse_distance2=0.1111111
reseau_300.elevation_angle1=np.arccos(0.7986355)

### Reseau 450 tpmm

In [14]:
reseau_450.line_density = 450/milli
reseau_450.inverse_distance1=0.1761099
reseau_450.inverse_distance2=0.1111111
reseau_450.elevation_angle1=np.arccos(0.7986355)
reseau_450.next = m2

### Reseau 600 tpmm

In [15]:
reseau_600.line_density = 600/milli
reseau_600.inverse_distance1=0.2002547
reseau_600.inverse_distance2=0.1111111
reseau_600.elevation_angle1=np.arccos(0.7986355)

### Reseau MCA

In [16]:
reseau_MCA.line_density = 1800/milli
# reseau_MCA.inverse_distance1=1e-9
# reseau_MCA.inverse_distance2=1e-7
# reseau_MCA.elevation_angle1=np.arccos(0.939693)
# Note : ce réseau n'est pas VLS

### M2


In [17]:
# l'angle et la distance sont calculés au moment de l'alignement du réseau
m2.phi = 180*degree # rad
m2.next = m3

### M3 

In [18]:
# la distance au M2-M3 dépend de l'énergie et est calculée à l'alignement
m3.phi = 90*degree # rad
m3.theta = (180-177.6)/2*degree # rad
m3.major_curvature = 1/83 # m-1
m3.minor_curvature = 1/0.1462124 # m-1
m3.next = plan_intermediaire

### Plan de focalisation intermédiaire 

In [19]:
plan_intermediaire.distance_from_previous = 0.2
plan_intermediaire.phi = -90*degree #rad
plan_intermediaire.next = fente

### Fente de sortie du mono

In [20]:
fente.distance_from_previous = 3.3
fente.recording_mode = RecordingMode.recording_output
fente.next = m5 # branche peem

### M5 

In [21]:
m5.distance_from_previous = 1
m5.phi = 90*degree
m5.curvature = 1/35 # m-1
m5.axis_angle = 0 # tangential cylinder 
m5.theta = 1.75*degree
m5.next = foc_intermediaire 

### Focalisation intermediaire 

In [22]:
foc_intermediaire.phi = -90*degree # rad
foc_intermediaire.distance_from_previous = 0.6187585
foc_intermediaire.next = diagnostic
foc_intermediaire.recording_mode = RecordingMode.recording_output

### Diagnostic faisceau

In [23]:
diagnostic.distance_from_previous = 7.7812242
diagnostic.recording_mode = RecordingMode.recording_output
diagnostic.next = m6

### M6 

In [24]:
m6.phi = -90*degree # rad
m6.theta = 1.75*degree # rad
m6.distance_from_previous = 0.6
m6.inverse_p = 1/(0.6 + 7.7812242) # m-1
m6.inverse_q = 1/(0.5 + 1.80) # m-1
m6.theta0 = 1.75*degree #rad
m6.next = m7

### M7

In [25]:
m7.distance_from_previous = 0.5
m7.phi = 90*degree # rad
m7.theta = 1.75*degree # rad
m7.inverse_p = 1/(0.6 + 7.7812242 + 0.5) # m-1
m7.inverse_q = 1/(1.80) # m-1
m7.theta0 = 1.75*degree #rad
m7.next = peem

### PEEM

In [26]:
peem.distance_from_previous = 1.80 # m
peem.recording_mode = RecordingMode.recording_output

### STXM

In [27]:
stxm.distance_from_previous = 1.45 # m
stxm.recording_mode = RecordingMode.recording_output

In [28]:
Hermes.active_chain = "BE_PEEM_450"
print(f.next)

Chaîne BE_PEEM_450:
	S_ONDUL_BE -> pupille -> M1A -> M1B -> foca -> f -> Reseau_450 -> plan -> M3tor -> planint -> Fente -> M5CylConCave -> FocIntHo -> M6Cc -> M7Cc -> PEEM 
Element Reseau_450 of class <class 'pyoptix.classes.PlaneHoloGrating'>
	 at 0.0 m from f
	 pointing to plan
	 oriented in pitch at 0.0 deg (deviation 180.0 deg)
	 oriented in roll at 0.0 deg
	 oriented in yaw at 0.0 deg



## Définition des scripts d'alignement

In [29]:
lambda_align = 6e-9
lambda_radiate = 6e-9

### Onduleurs

#### Données onduleur
L'onduleur basse énergie se situe à 1.849 m du centre de la section, l'onduleur haute énergie est centré.

dimension de la source électronique (sigmas en m)

SDM 169 X 6 µm ;    SDL 188 X 9 µm ;  SDC  321 x  8.1

dimension de la source électronique (sigmas en rd)

SDM 27 X 5 µrd ;    SDL  30 x 4.5 µrd; SDC 14.6 x 4.5  µrd

Hermes se situe dans une section droite moyenne de la maille

In [30]:
Sh = 169e-6 # m
Sv = 6e-6 # m
Sph = 27e-6
Spv = 5e-6
# Longueur de l'onduleur (m)
Lond = 1.8
# Position (m) du centre de la S D au centre de l'onduleur
# Csd = dist SD / l'ond. utilisé comme origine (sens + vers l'aval)
# Cond = dist ond. / la SD utilisée comme origine (sens + vers l'aval)
Csd_BE = 0
Cond_BE = 1.849
Csd_HE = 0
Cond_HE = 1.849
if Hermes.active_chain[0] == onduleur_BE:
    Csd = Csd_BE
    Cond = Cond_BE
    onduleur = onduleur_BE
else:
    Csd = Csd_HE
    Cond = Cond_HE
    onduleur = onduleur_HE
    
# Facteur d'accord de l'onduleur   = augmentation de la divergence  en cas de desaccord
Ka = 1.4

#### Elargissement dus aux erreur de pente 
ignorés pour l'instant

In [31]:
SSlV = 0
SSlH = 0
#SSlV = SlopeM1h * DistSM1 *  np.cos( M1dev /2.) + SlopeG1 * (DistSM1 + DistM1G ) )*2
# SSlH = $SlopeM1v *2 *  $DistSM1

#### Calculs des waists
non implémentés sous optix pour le moment

In [32]:
print(f"Lambda = {lambda_radiate/nano} nm (E = {hc/(lambda_radiate/nano):.2f} eV)")
Su2 = lambda_radiate * Lond / Ka/ (8 * pi**2) 
Spu2 = lambda_radiate * Ka / (2 * Lond) 
print(f"SigmaU =  {np.sqrt(Su2)}  SigmaUp = {np.sqrt(Spu2)} ")
if  Cond == 0 :
    Zv = np.sqrt(Spv) / ( Spv**2 + Spu2)  * Csd 
    Zh = Sph**2 / ( Sph**2 + Spu2)  * Csd 
else :
    Zv = Spu2 / ( Spv**2 + Spu2)  * Cond 
    Zh = Spu2 / ( Sph**2 + Spu2)  * Cond 

print(f"Zv = {Zv}      Zh = {Zh} ")
# onduleur.WaistZ = Zv
# onduleur.WaistY = Zh

Lambda = 6.0 nm (E = 206.64 eV)
SigmaU =  9.88446103441284e-06  SigmaUp = 4.8304589153964794e-05 
Zv = 1.829399293286219      Zh = 1.4088385762490476 


#### Taille et divergence de la source

In [33]:
sqv2 = 1. / ( 1./Spv**2 + 1./Spu2 ) 
sqh2 = 1. / ( 1./Sph**2 + 1./Spu2 ) 
if  Cond == 0:
    SigmaV = np.sqrt(( Sv**2 + SSlV**2 + Su2 + Csd**2 *sqv2))
    SigmaH = np.sqrt(( Sh**2 + SSlH**2 + Su2 + Csd**2 *sqh2))
else:
    SigmaV = np.sqrt(( Sv**2 + SSlV**2 + Su2 + Cond**2 *sqv2))
    SigmaH = np.sqrt(( Sh**2 + SSlH**2 + Su2 + Cond**2 *sqh2))
print(f"sigmaY = {SigmaH}      sigmaZ = {SigmaV} ")
onduleur.sigma_x = SigmaH
onduleur.sigma_y =  SigmaV

SigmaPV = np.sqrt((Spv**2 + Spu2)) 
SigmaPH = np.sqrt((Sph**2 + Spu2)) 
print(f"sigmaYp = {SigmaPH}       sigmaZp = {SigmaPV}")
onduleur.sigma_x_div = SigmaPH
onduleur.sigma_y_div = SigmaPV

pupille.distance_from_previous -= Cond - Csd # distance centre onduleur - centre source

sigmaY = 0.00017480762475497748      sigmaZ = 1.4773846901636054e-05 
sigmaYp = 5.533835318595353e-05       sigmaZp = 4.856267428111155e-05


### M1 

In [34]:
if m1a.next == m1b:
    m1a.theta = 2.5*degree # rad
else:
    m1a.theta = 1.2*degree # rad

### Réseaux

In [35]:
cff = 0.6
omega = 0.72*degree # rad
delta_z = 15*milli # m
deviation = np.nan

if cff > 1:
    cff = 1/cff # selon definition
if focalisation_horizontale.next == reseau_MCA: # cas omega constant
    deviation = 2*np.arccos(lambda_align*reseau_MCA.line_density/2./np.sin(omega))
    alpha = (pi-deviation)/2 + omega
    beta =  (pi-deviation)/2 - omega
    reseau_MCA.theta = alpha
else: 
    reseau = f.next # cas cff constant
    nu = lambda_align*reseau.line_density
    k = 1-cff**2
    X = (np.sqrt(k**2 + nu**2*cff**2) - nu) / k
    alpha = np.arccos(X)
    beta = np.arccos(nu + X)
    deviation = pi - alpha - beta
    reseau.theta = alpha
    print(f"k={k}, nu={nu}, alpha={alpha}, beta={beta}")

    
m2.theta = (pi - deviation)/2
dist = delta_z/np.cos(deviation/2)*(1-np.sin(deviation/2)/2)
m2.distance_from_previous = dist
m3.distance_from_previous = 0.55 - dist*np.cos(deviation)

print(f"Alpha = {alpha/degree} deg,")
print(f"Beta = {beta/degree} deg")
print(f"Deviation = {deviation/degree} deg")
print(f"Distance reseau-M2 = {dist}")
print(f"Distance M2-M3 = {0.55 + dist*np.cos(deviation)}")

k=0.64, nu=0.006, alpha=0.1369219880059969, beta=0.081988562380883
Alpha = 7.8450520352845 deg,
Beta = 4.697598592769668 deg
Deviation = 167.4573493719458 deg
Distance reseau-M2 = 0.06906901598354906
Distance M2-M3 = 0.48257934230733296


## Exécution de la simulation

In [36]:
print(Hermes.active_chain)
Hermes.align(lambda_align, Hermes.active_chain[0])
Hermes.clear_impacts()

S_ONDUL_BE -> pupille -> M1A -> M1B -> foca -> f -> Reseau_450 -> plan -> M3tor -> planint -> Fente -> M5CylConCave -> FocIntHo -> M6Cc -> M7Cc -> PEEM 


1

In [37]:
Hermes.generate(lambda_radiate)

5000

In [38]:
Hermes.radiate()

-1747556863

## Visualisations 

#### Résumé de l'alignement

In [39]:
for oe in Hermes.active_chain:
    print(oe)

Element S_ONDUL_BE of class <class 'pyoptix.classes.Source'>
	 at 0.0 m from None
	 pointing to pupille
	 oriented in pitch at 0.0 deg (deviation 180.0 deg)
	 oriented in roll at 0.0 deg
	 oriented in yaw at 0.0 deg

Element pupille of class <class 'pyoptix.classes.PlaneFilm'>
	 at 18.151 m from S_ONDUL_BE
	 pointing to M1A
	 oriented in pitch at 0.0 deg (deviation 180.0 deg)
	 oriented in roll at 0.0 deg
	 oriented in yaw at 0.0 deg

Element M1A of class <class 'pyoptix.classes.PlaneMirror'>
	 at 0.4783 m from pupille
	 pointing to M1B
	 oriented in pitch at 2.5 deg (deviation 175.0 deg)
	 oriented in roll at -90.0 deg
	 oriented in yaw at 0.0 deg

Element M1B of class <class 'pyoptix.classes.ToroidalMirror'>
	 at 3.2096 m from M1A
	 pointing to foca
	 oriented in pitch at 2.5 deg (deviation 175.0 deg)
	 oriented in roll at 180.0 deg
	 oriented in yaw at 0.0 deg

Element foca of class <class 'pyoptix.classes.PlaneFilm'>
	 at 3.2096 m from M1B
	 pointing to f
	 oriented in pitch at 0.0

#### Fente de sortie du monochromateur 

In [40]:
fente.show_diagram(Hermes.active_chain[0].nrays, distance_from_oe=100)

#### focalisation intermédiaire après M5 

In [41]:
foc_intermediaire.show_diagram(Hermes.active_chain[0].nrays)

#### Plan du PEEM

In [42]:
peem.show_diagram(Hermes.active_chain[0].nrays, show_first_rays=True)

          X         Y        dX        dY        Lambda
0 -0.865583 -0.233805 -0.257980 -0.096771  6.000000e-09
1 -0.442168  0.672865 -0.144007  0.185685  6.000000e-09
2  0.403156  0.115750  0.200614  0.067006  6.000000e-09
3  0.361441 -0.126900  0.162449 -0.074252  6.000000e-09
4 -0.857460 -0.272031 -0.254961 -0.108142  6.000000e-09


In [43]:
Hermes.draw_active_chain()

S_ONDUL_BE -> pupille -> M1A -> M1B -> foca -> f -> Reseau_450 -> plan -> M3tor -> planint -> Fente -> M5CylConCave -> FocIntHo -> M6Cc -> M7Cc -> PEEM 
